# 问题2：原材料订购与转运方案优化

## 问题分析

问题2要求解决两个核心问题：
1. 确定企业至少需要选择多少家供应商来满足生产需求
2. 制定未来24周最经济的原材料订购方案和损耗最少的转运方案

## 基础数据回顾

基于第一问的分析，我们已经获得：
- 402家供应商的供货特征量化分析
- 50家最重要供应商的排名
- 各供应商的供货稳定性、可靠性评估
- 转运商的损耗率分析

## 约束条件梳理

### 生产需求约束
- 企业每周产能：2.82万立方米
- 原材料消耗比例：
  - A类：0.6立方米/立方米产品
  - B类：0.66立方米/立方米产品  
  - C类：0.72立方米/立方米产品
- 库存要求：不少于满足两周生产需求的库存量

### 供应商约束
- 供应商实际供货量可能偏离订货量
- 需要基于历史数据预测供货稳定性
- 每家供应商有其供应能力上限

### 转运约束
- 每家转运商运输能力：6000立方米/周
- 转运过程中存在损耗（损耗率varies by转运商）
- 一家供应商每周的原材料尽量由一家转运商运输

### 成本约束
- A类原材料采购单价比C类高20%
- B类原材料采购单价比C类高10%
- 运输和储存的单位费用相同

## 解决思路

### 第一阶段：最少供应商数量确定

#### 方法1：基于需求量的理论计算
1. **计算每周最大原材料需求**
   - 按最高效率（A类原材料）：2.82万 × 0.6 = 1.692万立方米/周
   - 按最低效率（C类原材料）：2.82万 × 0.72 = 2.03万立方米/周

2. **考虑安全库存需求**
   - 两周生产需求作为安全库存
   - 总需求 = 周需求 + 安全库存需求

3. **基于TOP50供应商的供应能力**
   - 从第一问结果中提取TOP50供应商的平均供货能力
   - 按供应能力从高到低排序，累计计算直到满足总需求

#### 方法2：基于历史数据的蒙特卡洛模拟
1. **建立供应商供货概率模型**
   - 基于历史数据建立每个供应商的供货量分布
   - 考虑供货的不确定性和波动性

2. **多次模拟验证**
   - 随机选择不同数量的供应商组合
   - 模拟24周的供货情况
   - 计算满足需求的概率，确定最少供应商数量

### 第二阶段：最经济订购方案制定

#### 目标函数设计
$$\text{最小化总成本} = \text{采购成本} + \text{库存成本} + \text{缺货惩罚成本}$$

其中：
- 采购成本 = Σ(订货量 × 单价 × 材料类型系数)
- 库存成本 = Σ(库存量 × 单位库存费用)
- 缺货惩罚成本 = Σ(缺货量 × 单位缺货惩罚)

#### 决策变量
- $x_{i,t}$：第t周向供应商i的订货量
- $I_{t}$：第t周末的库存量
- $S_{t}$：第t周的缺货量

#### 约束条件
1. **需求满足约束**：$\text{库存} + \text{本周接收量} - \text{本周消耗} \geq 0$

2. **库存平衡约束**：$I_t = I_{t-1} + \text{接收量}_t - \text{需求}_t$

3. **最小库存约束**：$I_t \geq 2 \times \text{周需求}$

4. **供应商产能约束**：$x_{i,t} \leq \text{供应商i的最大供应能力}$

5. **非负约束**：$x_{i,t} \geq 0$

### 第三阶段：损耗最少转运方案

#### 目标函数
$$\text{最小化总损耗} = \sum_{i,j,t} (\text{供货量}_{i,t} \times \text{损耗率}_{j,t} \times y_{i,j,t})$$

#### 决策变量
- $y_{i,j,t}$：第t周供应商i是否由转运商j运输（0-1变量）

#### 约束条件
1. **转运商选择约束**：$\sum_j y_{i,j,t} \leq 1$ （每个供应商每周最多选择一个转运商）

2. **转运能力约束**：$\sum_i (\text{供货量}_{i,t} \times y_{i,j,t}) \leq 6000$ （每个转运商的周运输能力）

3. **逻辑约束**：只有当供应商i在第t周有供货时，才能选择转运商

## 求解策略

### 整体求解框架
1. **两阶段优化方法**
   - 第一阶段：固定供应商选择，优化订购量
   - 第二阶段：基于订购方案，优化转运分配

2. **启发式算法**
   - 贪心算法：优先选择性价比高的供应商和转运商
   - 遗传算法：用于供应商组合优化
   - 动态规划：用于时间序列的订购量优化

### 模型验证
1. **历史数据回测**：用前期数据验证模型的有效性
2. **敏感性分析**：分析关键参数变化对结果的影响
3. **鲁棒性测试**：测试方案在不确定环境下的表现

## 预期分析内容

### 订购方案分析
1. **供应商选择结果**：最终选择的供应商数量和名单
2. **订购量分布**：各周、各供应商的具体订购量
3. **成本分析**：总成本构成和每周成本变化
4. **风险评估**：供货风险和库存风险分析

### 转运方案分析
1. **转运商利用率**：各转运商的使用频率和运输量
2. **损耗分析**：总损耗量和损耗率统计
3. **运输效率**：运输能力利用率分析

### 实施效果评估
1. **需求满足率**：各周需求满足程度
2. **库存水平**：库存量变化趋势和安全性
3. **经济效益**：成本节约和效率提升
4. **风险控制**：供应风险的有效控制程度

## 技术实施方案

### 数据预处理
1. **利用第一问的分析结果**
   - 导入供应商可靠性评估数据
   - 导入转运商损耗率分析数据
   - 导入TOP50重要供应商名单

2. **建立基础数据结构**
   - 供应商基本信息（名称、类型、历史供货数据）
   - 转运商基本信息（名称、历史损耗率数据）
   - 生产需求参数（产能、原材料消耗比例、成本系数）

### 核心算法设计

#### 算法1：最少供应商数量确定
```
输入：TOP50供应商数据、生产需求、安全库存要求
输出：最少供应商数量N

步骤：
1. 按供应商重要性排序（来自第一问结果）
2. 计算累计供应能力
3. 考虑供货不确定性，添加安全边际
4. 返回满足需求的最少供应商数量
```

#### 算法2：订购方案优化
```
输入：选定供应商集合、24周需求预测、成本参数
输出：最优订购计划矩阵

方法：多目标线性规划
- 目标1：最小化总成本
- 目标2：最大化供应稳定性
- 约束：需求满足、库存要求、供应能力限制
```

#### 算法3：转运方案优化
```
输入：订购方案、转运商损耗数据、运输能力
输出：最优转运分配方案

方法：匈牙利算法 + 启发式优化
- 构建成本矩阵（基于损耗率）
- 考虑运输能力约束
- 优化供应商-转运商匹配
```

### 关键技术点

#### 1. 不确定性处理
- **供货量预测**：基于历史数据的时间序列分析
- **需求波动**：考虑生产计划的不确定性
- **应急预案**：设计供应商备选方案

#### 2. 多目标优化
- **成本与风险平衡**：帕累托前沿分析
- **权重设定**：基于企业决策偏好
- **方案比较**：多准则决策分析（MCDM）

#### 3. 滚动优化策略
- **周期性重新规划**：每4周更新一次方案
- **反馈机制**：根据实际执行情况调整参数
- **学习能力**：历史数据不断更新模型参数

## 代码实现框架

### 主要模块划分
1. **数据加载模块**：`data_loader.py`
2. **需求预测模块**：`demand_forecast.py`  
3. **供应商选择模块**：`supplier_selection.py`
4. **订购优化模块**：`order_optimization.py`
5. **转运优化模块**：`transport_optimization.py`
6. **结果分析模块**：`result_analysis.py`
7. **可视化模块**：`visualization.py`

### 优化求解工具
- **线性规划**：PuLP库或Gurobi
- **整数规划**：用于0-1决策变量
- **启发式算法**：遗传算法（DEAP库）
- **统计分析**：SciPy、Statsmodels

### 输出结果格式
1. **订购方案表**：供应商 × 周次 矩阵
2. **转运方案表**：供应商-转运商分配表
3. **成本分析报告**：各项成本明细和趋势
4. **风险评估报告**：供应风险和应对措施
5. **可视化图表**：成本趋势、库存变化、供应商分布等

## 评估指标体系

### 经济性指标
- **总成本**：采购成本 + 运输成本 + 库存成本
- **单位成本**：每立方米产品的原材料成本
- **成本波动性**：成本标准差和变异系数

### 稳定性指标  
- **需求满足率**：实际满足需求/计划需求
- **库存充足率**：满足安全库存要求的周数比例
- **供应商集中度**：基尼系数或HHI指数

### 效率性指标
- **损耗率**：总损耗量/总供货量
- **运输利用率**：实际运输量/运输能力
- **库存周转率**：年消耗量/平均库存量

### 风险性指标
- **供应中断风险**：供应商失效概率
- **价格波动风险**：成本变异系数
- **需求冲击风险**：极端情况下的应对能力

## 具体实施步骤

### Step 1: 数据整合与预处理
```python
# 伪代码示例
def load_problem1_results():
    """加载第一问的分析结果"""
    top50_suppliers = pd.read_excel('TOP50最重要供应商.xlsx')
    reliability_data = pd.read_excel('供应商可靠性评估.xlsx') 
    transporter_analysis = pd.read_excel('转运商分析结果.xlsx')
    return top50_suppliers, reliability_data, transporter_analysis

def calculate_demand_parameters():
    """计算生产需求参数"""
    weekly_production = 28200  # 立方米/周
    material_ratios = {'A': 0.6, 'B': 0.66, 'C': 0.72}
    price_multipliers = {'A': 1.2, 'B': 1.1, 'C': 1.0}
    safety_stock_weeks = 2
    return weekly_production, material_ratios, price_multipliers, safety_stock_weeks
```

### Step 2: 最少供应商数量确定
```python
# 伪代码示例
def determine_minimum_suppliers(suppliers_data, demand_params):
    """确定最少供应商数量"""
    # 方法1: 确定性分析
    min_suppliers_deterministic = calculate_deterministic_minimum()
    
    # 方法2: 随机性分析（考虑供货不确定性）
    min_suppliers_stochastic = monte_carlo_simulation()
    
    # 方法3: 风险调整分析  
    min_suppliers_risk_adjusted = risk_adjusted_calculation()
    
    return max(min_suppliers_deterministic, min_suppliers_stochastic, min_suppliers_risk_adjusted)
```

### Step 3: 订购方案优化建模
```python
# 伪代码示例
def optimize_ordering_plan(selected_suppliers, weeks=24):
    """优化订购方案"""
    model = pulp.LpProblem("Material_Ordering", pulp.LpMinimize)
    
    # 决策变量：x[i][t] = 第t周向供应商i的订货量
    x = {}
    for supplier in selected_suppliers:
        for week in range(1, weeks+1):
            x[supplier, week] = pulp.LpVariable(f"Order_{supplier}_{week}", lowBound=0)
    
    # 目标函数：最小化总成本
    total_cost = calculate_total_cost_expression(x)
    model += total_cost
    
    # 约束条件
    add_demand_constraints(model, x)
    add_inventory_constraints(model, x)
    add_capacity_constraints(model, x)
    
    # 求解
    model.solve()
    return extract_solution(x)
```

### Step 4: 转运方案优化
```python
# 伪代码示例  
def optimize_transport_plan(ordering_plan, transporters_data):
    """优化转运方案"""
    transport_model = pulp.LpProblem("Transport_Assignment", pulp.LpMinimize)
    
    # 决策变量：y[i][j][t] = 第t周供应商i是否由转运商j运输
    y = {}
    for supplier in suppliers:
        for transporter in transporters:
            for week in range(1, 25):
                y[supplier, transporter, week] = pulp.LpVariable(
                    f"Transport_{supplier}_{transporter}_{week}", cat='Binary')
    
    # 目标函数：最小化总损耗
    total_loss = calculate_total_loss_expression(y)
    transport_model += total_loss
    
    # 约束条件
    add_assignment_constraints(transport_model, y)  # 每个供应商每周最多一个转运商
    add_capacity_constraints(transport_model, y)    # 转运商能力限制
    
    # 求解
    transport_model.solve()
    return extract_transport_solution(y)
```

### Step 5: 结果分析与验证
```python
# 伪代码示例
def analyze_solution_performance(ordering_plan, transport_plan):
    """分析方案性能"""
    # 经济性分析
    total_cost_analysis = calculate_cost_breakdown(ordering_plan)
    cost_comparison = compare_with_baseline(total_cost_analysis)
    
    # 稳定性分析  
    demand_satisfaction = simulate_demand_satisfaction(ordering_plan)
    inventory_analysis = analyze_inventory_levels(ordering_plan)
    
    # 风险分析
    supply_risk_assessment = assess_supply_risks(ordering_plan)
    sensitivity_analysis = perform_sensitivity_analysis()
    
    return {
        'cost_analysis': total_cost_analysis,
        'performance_metrics': demand_satisfaction,
        'risk_assessment': supply_risk_assessment
    }
```

## 预期结果与输出

### 1. 核心数值结果
- **最少供应商数量**：基于计算得出的具体数值（例如：35家）
- **24周订购计划表**：详细的订购量分配矩阵
- **转运分配方案**：供应商与转运商的最优匹配

### 2. 经济效益分析
- **总成本预估**：24周期间的总采购和运输成本
- **成本结构分析**：
  - 原材料采购成本占比
  - 运输成本占比  
  - 库存持有成本占比
- **与基准方案对比**：成本节约额度和节约率

### 3. 方案可行性验证
- **需求满足率**：≥ 95%（目标值）
- **库存安全性**：安全库存达成率 ≥ 90%
- **供应商利用率**：各供应商的使用频率分布
- **转运商负荷率**：运输能力利用效率

### 4. 风险控制效果
- **供应集中度**：避免过度依赖少数供应商
- **应急响应能力**：关键供应商断供时的应对方案
- **成本波动控制**：成本变异系数控制在合理范围

### 5. 可视化展示
- **订购量时间序列图**：展示24周的订购量变化趋势
- **供应商贡献度饼图**：各供应商供货量占比
- **转运商使用热力图**：转运商在不同时间的使用频率
- **成本构成堆积图**：各项成本的时间变化

### 6. 敏感性分析结果
- **需求变化影响**：±10%需求变化对方案的影响
- **供应商产能变化**：关键供应商产能下降的应对
- **转运成本变化**：运输费用波动的影响分析


## 代码实现：最少供应商数量的蒙特卡洛模拟

下面实现具体的蒙特卡洛模拟代码来确定最少供应商数量：

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体和图表样式
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print("库导入完成，开始加载第一问的分析结果...")

In [ ]:
# 加载第一问的分析结果
def load_problem1_results():
    """加载第一问的分析结果"""
    try:
        # 加载TOP50重要供应商数据
        top50_suppliers = pd.read_excel('DataFrames/TOP50最重要供应商_20250802_142429.xlsx')
        
        # 加载供应商可靠性评估数据
        reliability_files = [
            'DataFrames/供应商可靠性评估_20250802_142426.xlsx',
            'DataFrames/供应商可靠性评估_20250802_142428.xlsx'
        ]
        
        reliability_data = []
        for file in reliability_files:
            try:
                df = pd.read_excel(file)
                reliability_data.append(df)
            except:
                pass
        
        # 加载原始供应商数据
        supplier_order = pd.read_excel('C/附件1 近5年402家供应商的相关数据.xlsx', sheet_name='企业的订货量')
        supplier_supply = pd.read_excel('C/附件1 近5年402家供应商的相关数据.xlsx', sheet_name='供应商的供货量')
        
        print("✓ 数据加载完成")
        return top50_suppliers, reliability_data, supplier_order, supplier_supply
    
    except Exception as e:
        print(f"数据加载出错: {e}")
        return None, None, None, None

# 设置生产需求参数
def set_production_parameters():
    """设置生产需求参数"""
    params = {
        'weekly_production': 28200,  # 立方米/周
        'material_ratios': {'A': 0.6, 'B': 0.66, 'C': 0.72},  # 每立方米产品需要的原材料
        'price_multipliers': {'A': 1.2, 'B': 1.1, 'C': 1.0},  # 相对C类的价格倍数
        'safety_stock_weeks': 2,  # 安全库存周数
        'planning_weeks': 24  # 规划周数
    }
    
    print("生产参数设置:")
    print(f"  每周产能: {params['weekly_production']:,} 立方米")
    print(f"  安全库存要求: {params['safety_stock_weeks']} 周")
    print(f"  规划期长度: {params['planning_weeks']} 周")
    
    return params

# 执行数据加载
top50_suppliers, reliability_data, supplier_order, supplier_supply = load_problem1_results()
production_params = set_production_parameters()

In [ ]:
# 分析供应商历史供货特征
def analyze_supplier_characteristics(supplier_supply):
    """分析供应商的历史供货特征，为蒙特卡洛模拟做准备"""
    
    if supplier_supply is None:
        print("供应商数据未加载成功")
        return None
    
    # 获取供应商基本信息
    suppliers_info = supplier_supply.iloc[:, :2].copy()
    suppliers_info.columns = ['供应商名称', '材料类型']
    
    # 获取供货量数据（从第3列开始的240周数据）
    supply_data = supplier_supply.iloc[:, 2:].values
    
    # 计算每个供应商的统计特征
    supplier_stats = []
    
    for i, (name, material_type) in enumerate(suppliers_info.values):
        weekly_supply = supply_data[i]
        
        # 过滤掉0值，只分析实际供货的周次
        non_zero_supply = weekly_supply[weekly_supply > 0]
        
        if len(non_zero_supply) > 0:
            stats_dict = {
                '供应商名称': name,
                '材料类型': material_type,
                '平均供货量': np.mean(non_zero_supply),
                '供货量标准差': np.std(non_zero_supply),
                '最大供货量': np.max(non_zero_supply),
                '最小供货量': np.min(non_zero_supply),
                '供货频率': len(non_zero_supply) / len(weekly_supply),  # 实际供货周数/总周数
                '供货稳定性': 1 - (np.std(non_zero_supply) / np.mean(non_zero_supply)) if np.mean(non_zero_supply) > 0 else 0,
                '供货能力评分': np.mean(non_zero_supply) * (len(non_zero_supply) / len(weekly_supply))
            }
        else:
            stats_dict = {
                '供应商名称': name,
                '材料类型': material_type,
                '平均供货量': 0,
                '供货量标准差': 0,
                '最大供货量': 0,
                '最小供货量': 0,
                '供货频率': 0,
                '供货稳定性': 0,
                '供货能力评分': 0
            }
        
        supplier_stats.append(stats_dict)
    
    df_stats = pd.DataFrame(supplier_stats)
    
    print(f"供应商历史供货特征分析完成:")
    print(f"  总供应商数量: {len(df_stats)}")
    print(f"  有效供应商数量: {len(df_stats[df_stats['平均供货量'] > 0])}")
    print(f"  A类材料供应商: {len(df_stats[df_stats['材料类型'] == 'A'])}")
    print(f"  B类材料供应商: {len(df_stats[df_stats['材料类型'] == 'B'])}")
    print(f"  C类材料供应商: {len(df_stats[df_stats['材料类型'] == 'C'])}")
    
    return df_stats

# 执行供应商特征分析
supplier_characteristics = analyze_supplier_characteristics(supplier_supply)

In [ ]:
# 计算生产需求
def calculate_demand_requirements(production_params):
    """计算各种情况下的原材料需求"""
    
    weekly_production = production_params['weekly_production']
    material_ratios = production_params['material_ratios']
    safety_weeks = production_params['safety_stock_weeks']
    
    # 计算每种材料类型的周需求量
    demand_scenarios = {}
    
    for material_type, ratio in material_ratios.items():
        weekly_demand = weekly_production * ratio
        safety_stock = weekly_demand * safety_weeks
        total_demand_per_week = weekly_demand + safety_stock / production_params['planning_weeks']
        
        demand_scenarios[material_type] = {
            '周生产需求': weekly_demand,
            '安全库存需求': safety_stock,
            '平均周需求': total_demand_per_week,
            '24周总需求': weekly_demand * production_params['planning_weeks'] + safety_stock
        }
    
    print("各材料类型需求分析:")
    for material_type, demands in demand_scenarios.items():
        print(f"\n{material_type}类材料:")
        for key, value in demands.items():
            print(f"  {key}: {value:,.0f} 立方米")
    
    return demand_scenarios

# 筛选和排序供应商
def prepare_suppliers_for_simulation(supplier_characteristics, top50_suppliers=None):
    """为蒙特卡洛模拟准备供应商数据"""
    
    if supplier_characteristics is None:
        return None
    
    # 过滤有效供应商（有实际供货记录的）
    valid_suppliers = supplier_characteristics[
        (supplier_characteristics['平均供货量'] > 0) & 
        (supplier_characteristics['供货频率'] > 0.1)  # 至少10%的时间有供货
    ].copy()
    
    # 按供货能力评分排序
    valid_suppliers = valid_suppliers.sort_values('供货能力评分', ascending=False)
    
    # 为每个供应商添加风险调整因子
    valid_suppliers['风险调整因子'] = (
        valid_suppliers['供货稳定性'] * 0.4 + 
        valid_suppliers['供货频率'] * 0.6
    )
    
    # 按材料类型分组
    suppliers_by_type = {}
    for material_type in ['A', 'B', 'C']:
        suppliers_by_type[material_type] = valid_suppliers[
            valid_suppliers['材料类型'] == material_type
        ].copy()
    
    print("供应商筛选结果:")
    for material_type, df in suppliers_by_type.items():
        print(f"  {material_type}类有效供应商: {len(df)}家")
        if len(df) > 0:
            print(f"    平均供货能力: {df['平均供货量'].mean():.0f} 立方米")
            print(f"    最大供货能力: {df['平均供货量'].max():.0f} 立方米")
    
    return valid_suppliers, suppliers_by_type

# 执行需求计算和供应商准备
demand_requirements = calculate_demand_requirements(production_params)
valid_suppliers, suppliers_by_type = prepare_suppliers_for_simulation(supplier_characteristics, top50_suppliers)

In [ ]:
# 蒙特卡洛模拟核心函数
def monte_carlo_supplier_simulation(suppliers_by_type, demand_requirements, 
                                   n_simulations=1000, confidence_level=0.95):
    """
    使用蒙特卡洛方法确定最少供应商数量
    
    参数:
    - suppliers_by_type: 按材料类型分组的供应商数据
    - demand_requirements: 需求要求
    - n_simulations: 模拟次数
    - confidence_level: 置信水平
    """
    
    results = {}
    
    for material_type in ['A', 'B', 'C']:
        print(f"\n=== {material_type}类材料蒙特卡洛模拟 ===")
        
        suppliers = suppliers_by_type[material_type]
        if len(suppliers) == 0:
            print(f"警告: 没有{material_type}类材料供应商")
            continue
            
        demand = demand_requirements[material_type]['24周总需求']
        print(f"目标需求: {demand:,.0f} 立方米")
        
        # 测试不同的供应商数量
        max_suppliers = min(len(suppliers), 50)  # 最多测试50家供应商
        success_rates = []
        
        for n_suppliers in range(1, max_suppliers + 1):
            selected_suppliers = suppliers.head(n_suppliers)
            success_count = 0
            
            # 进行蒙特卡洛模拟
            for sim in range(n_simulations):
                total_supply = 0
                
                for idx, supplier in selected_suppliers.iterrows():
                    # 基于历史数据建立供货量分布
                    mean_supply = supplier['平均供货量']
                    std_supply = supplier['供货量标准差']
                    risk_factor = supplier['风险调整因子']
                    
                    # 考虑24周的总供货量
                    expected_weeks = 24 * supplier['供货频率'] * risk_factor
                    
                    # 使用正态分布模拟单周供货量，然后乘以预期供货周数
                    if std_supply > 0:
                        weekly_supply = np.random.normal(mean_supply, std_supply)
                        weekly_supply = max(0, weekly_supply)  # 确保非负
                    else:
                        weekly_supply = mean_supply
                    
                    # 总供货量 = 周供货量 × 预期供货周数
                    supplier_total = weekly_supply * expected_weeks
                    total_supply += supplier_total
                
                # 检查是否满足需求
                if total_supply >= demand:
                    success_count += 1
            
            success_rate = success_count / n_simulations
            success_rates.append(success_rate)
            
            # 如果成功率达到置信水平，记录并可以提前结束
            if success_rate >= confidence_level:
                min_suppliers = n_suppliers
                print(f"  {n_suppliers:2d}家供应商: 成功率 {success_rate:.1%}")
                break
            else:
                if n_suppliers <= 10 or n_suppliers % 5 == 0:  # 减少输出
                    print(f"  {n_suppliers:2d}家供应商: 成功率 {success_rate:.1%}")
        
        # 记录结果
        results[material_type] = {
            '最少供应商数量': min_suppliers if 'min_suppliers' in locals() else max_suppliers,
            '最终成功率': success_rates[-1] if success_rates else 0,
            '成功率序列': success_rates,
            '总需求': demand,
            '可选供应商总数': len(suppliers)
        }
        
        print(f"结果: {material_type}类材料至少需要 {results[material_type]['最少供应商数量']} 家供应商")
        print(f"对应成功率: {results[material_type]['最终成功率']:.1%}")
    
    return results

# 执行蒙特卡洛模拟
print("开始蒙特卡洛模拟分析...")
print("模拟参数: 1000次模拟, 95%置信水平")

mc_results = monte_carlo_supplier_simulation(
    suppliers_by_type, 
    demand_requirements, 
    n_simulations=1000, 
    confidence_level=0.95
)

In [ ]:
# 混合材料类型的供应商组合优化
def mixed_material_simulation(valid_suppliers, production_params, 
                             n_simulations=1000, confidence_level=0.95):
    """
    考虑混合材料类型的供应商组合蒙特卡洛模拟
    在实际生产中，可以混合使用不同类型的原材料
    """
    
    print("\n=== 混合材料类型供应商组合优化 ===")
    
    # 计算总的原材料等价需求
    weekly_production = production_params['weekly_production']
    safety_weeks = production_params['safety_stock_weeks']
    planning_weeks = production_params['planning_weeks']
    
    # 基准需求（以C类材料为基准，因为它需求量最大）
    base_weekly_demand = weekly_production * 0.72  # C类材料比例
    safety_stock = base_weekly_demand * safety_weeks
    total_demand = base_weekly_demand * planning_weeks + safety_stock
    
    print(f"总原材料需求（C类等价）: {total_demand:,.0f} 立方米")
    
    # 材料转换系数（相对于C类）
    conversion_factors = {'A': 0.72/0.6, 'B': 0.72/0.66, 'C': 1.0}
    
    # 按综合评分排序所有供应商
    suppliers_ranked = valid_suppliers.copy()
    
    # 计算综合评分（考虑材料类型的效率）
    suppliers_ranked['材料效率'] = suppliers_ranked['材料类型'].map(conversion_factors)
    suppliers_ranked['综合评分'] = (
        suppliers_ranked['供货能力评分'] * 
        suppliers_ranked['材料效率'] * 
        suppliers_ranked['风险调整因子']
    )
    
    suppliers_ranked = suppliers_ranked.sort_values('综合评分', ascending=False)
    
    print(f"参与模拟的有效供应商总数: {len(suppliers_ranked)}")
    
    # 测试不同的供应商数量组合
    max_suppliers = min(len(suppliers_ranked), 60)  # 最多测试60家
    success_rates = []
    optimal_combinations = []
    
    for n_suppliers in range(5, max_suppliers + 1, 2):  # 从5家开始，每次增加2家
        selected_suppliers = suppliers_ranked.head(n_suppliers)
        success_count = 0
        combination_details = []
        
        # 进行蒙特卡洛模拟
        for sim in range(n_simulations):
            total_supply_equivalent = 0  # C类等价供应量
            
            for idx, supplier in selected_suppliers.iterrows():
                # 供货量模拟
                mean_supply = supplier['平均供货量']
                std_supply = supplier['供货量标准差']
                risk_factor = supplier['风险调整因子']
                material_efficiency = supplier['材料效率']
                
                # 预期供货周数
                expected_weeks = planning_weeks * supplier['供货频率'] * risk_factor
                
                # 模拟周供货量
                if std_supply > 0:
                    weekly_supply = np.random.normal(mean_supply, std_supply)
                    weekly_supply = max(0, weekly_supply)
                else:
                    weekly_supply = mean_supply
                
                # 总供货量转换为C类等价
                supplier_total = weekly_supply * expected_weeks * material_efficiency
                total_supply_equivalent += supplier_total
            
            # 检查是否满足需求
            if total_supply_equivalent >= total_demand:
                success_count += 1
            
            # 记录前100次模拟的详细信息用于分析
            if sim < 100:
                combination_details.append({
                    '模拟次数': sim + 1,
                    '总供应量': total_supply_equivalent,
                    '需求满足': total_supply_equivalent >= total_demand
                })
        
        success_rate = success_count / n_simulations
        success_rates.append(success_rate)
        
        # 记录供应商组合信息
        material_counts = selected_suppliers['材料类型'].value_counts()
        combination_info = {
            '供应商数量': n_suppliers,
            '成功率': success_rate,
            'A类供应商': material_counts.get('A', 0),
            'B类供应商': material_counts.get('B', 0),
            'C类供应商': material_counts.get('C', 0),
            '平均供应能力': selected_suppliers['平均供货量'].sum(),
            '风险调整后供应能力': (selected_suppliers['平均供货量'] * 
                                  selected_suppliers['风险调整因子']).sum()
        }
        optimal_combinations.append(combination_info)
        
        # 输出进度
        if n_suppliers <= 15 or n_suppliers % 10 == 0:
            print(f"  {n_suppliers:2d}家供应商 (A:{material_counts.get('A', 0)} B:{material_counts.get('B', 0)} C:{material_counts.get('C', 0)}): "
                  f"成功率 {success_rate:.1%}")
        
        # 如果成功率达到要求，可以提前结束
        if success_rate >= confidence_level:
            min_mixed_suppliers = n_suppliers
            break
    
    # 结果分析
    if 'min_mixed_suppliers' in locals():
        optimal_result = optimal_combinations[optimal_combinations.index(
            next(combo for combo in optimal_combinations if combo['供应商数量'] == min_mixed_suppliers)
        )]
    else:
        # 选择成功率最高的组合
        best_combo_idx = np.argmax([combo['成功率'] for combo in optimal_combinations])
        optimal_result = optimal_combinations[best_combo_idx]
        min_mixed_suppliers = optimal_result['供应商数量']
    
    print(f"\n混合材料优化结果:")
    print(f"  最少供应商数量: {min_mixed_suppliers}家")
    print(f"  成功率: {optimal_result['成功率']:.1%}")
    print(f"  供应商构成: A类{optimal_result['A类供应商']}家, "
          f"B类{optimal_result['B类供应商']}家, C类{optimal_result['C类供应商']}家")
    
    return {
        '最少供应商数量': min_mixed_suppliers,
        '最优组合': optimal_result,
        '所有组合': optimal_combinations,
        '成功率序列': success_rates,
        '最优供应商列表': suppliers_ranked.head(min_mixed_suppliers)
    }

# 执行混合材料类型模拟
mixed_results = mixed_material_simulation(
    valid_suppliers, 
    production_params, 
    n_simulations=1000,
    confidence_level=0.95
)

In [ ]:
# 结果可视化和分析
def visualize_simulation_results(mc_results, mixed_results):
    """可视化蒙特卡洛模拟结果"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('蒙特卡洛模拟结果分析', fontsize=16, fontweight='bold')
    
    # 1. 各材料类型最少供应商数量对比
    ax1 = axes[0, 0]
    materials = []
    min_suppliers = []
    success_rates = []
    
    for material_type, result in mc_results.items():
        materials.append(material_type + '类')
        min_suppliers.append(result['最少供应商数量'])
        success_rates.append(result['最终成功率'])
    
    # 添加混合材料结果
    materials.append('混合优化')
    min_suppliers.append(mixed_results['最少供应商数量'])
    success_rates.append(mixed_results['最优组合']['成功率'])
    
    bars = ax1.bar(materials, min_suppliers, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    ax1.set_title('各方案最少供应商数量对比')
    ax1.set_ylabel('供应商数量')
    
    # 在柱状图上添加成功率标签
    for i, (bar, rate) in enumerate(zip(bars, success_rates)):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{rate:.1%}', ha='center', va='bottom', fontweight='bold')
    
    # 2. 混合材料方案的供应商数量vs成功率
    ax2 = axes[0, 1]
    combinations = mixed_results['所有组合']
    supplier_counts = [combo['供应商数量'] for combo in combinations]
    success_rates_mixed = [combo['成功率'] for combo in combinations]
    
    ax2.plot(supplier_counts, success_rates_mixed, 'o-', linewidth=2, markersize=6, color='#FF6B6B')
    ax2.axhline(y=0.95, color='red', linestyle='--', alpha=0.7, label='95%置信线')
    ax2.axvline(x=mixed_results['最少供应商数量'], color='green', linestyle='--', alpha=0.7, label='最优数量')
    ax2.set_title('混合方案：供应商数量 vs 成功率')
    ax2.set_xlabel('供应商数量')
    ax2.set_ylabel('成功率')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # 3. 混合方案供应商类型构成
    ax3 = axes[1, 0]
    optimal_combo = mixed_results['最优组合']
    material_counts = [optimal_combo['A类供应商'], optimal_combo['B类供应商'], optimal_combo['C类供应商']]
    material_labels = ['A类材料', 'B类材料', 'C类材料']
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    
    wedges, texts, autotexts = ax3.pie(material_counts, labels=material_labels, colors=colors, 
                                       autopct='%1.0f%%', startangle=90)
    ax3.set_title(f'最优方案供应商构成\n(总计{mixed_results["最少供应商数量"]}家)')
    
    # 4. 需求满足分析
    ax4 = axes[1, 1]
    scenarios = ['单一A类', '单一B类', '单一C类', '混合优化']
    total_demands = []
    min_suppliers_all = []
    
    for material_type in ['A', 'B', 'C']:
        if material_type in mc_results:
            total_demands.append(mc_results[material_type]['总需求'])
            min_suppliers_all.append(mc_results[material_type]['最少供应商数量'])
        else:
            total_demands.append(0)
            min_suppliers_all.append(0)
    
    # 混合优化的等价需求
    total_demands.append(production_params['weekly_production'] * 0.72 * 
                        (production_params['planning_weeks'] + production_params['safety_stock_weeks']))
    min_suppliers_all.append(mixed_results['最少供应商数量'])
    
    # 计算效率指标（需求/供应商数量）
    efficiency = [demand/suppliers if suppliers > 0 else 0 
                 for demand, suppliers in zip(total_demands, min_suppliers_all)]
    
    bars = ax4.bar(scenarios, efficiency, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    ax4.set_title('各方案供应效率对比')
    ax4.set_ylabel('效率指标 (需求量/供应商数)')
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('Pictures/monte_carlo_simulation_results.svg', dpi=300, bbox_inches='tight')
    plt.show()

# 生成详细分析报告
def generate_analysis_report(mc_results, mixed_results, valid_suppliers):
    """生成详细的分析报告"""
    
    print("\n" + "="*60)
    print("蒙特卡洛模拟分析报告")
    print("="*60)
    
    print("\n📊 模拟结果总览:")
    print("-" * 40)
    
    # 单一材料类型结果
    for material_type, result in mc_results.items():
        print(f"\n{material_type}类材料单独供应:")
        print(f"  最少供应商数量: {result['最少供应商数量']}家")
        print(f"  成功率: {result['最终成功率']:.1%}")
        print(f"  总需求: {result['总需求']:,.0f} 立方米")
        print(f"  可选供应商: {result['可选供应商总数']}家")
    
    # 混合材料结果
    print(f"\n混合材料优化方案:")
    optimal_combo = mixed_results['最优组合']
    print(f"  最少供应商数量: {optimal_combo['供应商数量']}家")
    print(f"  成功率: {optimal_combo['成功率']:.1%}")
    print(f"  A类供应商: {optimal_combo['A类供应商']}家")
    print(f"  B类供应商: {optimal_combo['B类供应商']}家") 
    print(f"  C类供应商: {optimal_combo['C类供应商']}家")
    print(f"  总供应能力: {optimal_combo['平均供应能力']:,.0f} 立方米")
    
    print("\n📈 关键发现:")
    print("-" * 40)
    
    # 找出最优方案
    all_options = []
    for material_type, result in mc_results.items():
        all_options.append((f"{material_type}类单一供应", result['最少供应商数量'], result['最终成功率']))
    all_options.append(("混合材料优化", optimal_combo['供应商数量'], optimal_combo['成功率']))
    
    # 按供应商数量排序
    all_options.sort(key=lambda x: x[1])
    
    print(f"1. 最经济方案: {all_options[0][0]} ({all_options[0][1]}家供应商)")
    print(f"2. 最高成功率方案: {max(all_options, key=lambda x: x[2])[0]} (成功率{max(all_options, key=lambda x: x[2])[2]:.1%})")
    
    # 风险分析
    if optimal_combo['成功率'] >= 0.95:
        risk_level = "低风险"
    elif optimal_combo['成功率'] >= 0.90:
        risk_level = "中等风险"
    else:
        risk_level = "高风险"
    
    print(f"3. 混合方案风险等级: {risk_level}")
    
    # 供应商利用率分析
    optimal_suppliers = mixed_results['最优供应商列表']
    avg_capacity_utilization = optimal_suppliers['供货频率'].mean()
    print(f"4. 平均供应商利用率: {avg_capacity_utilization:.1%}")
    
    print("\n💡 建议:")
    print("-" * 40)
    
    recommendations = []
    
    if mixed_results['最少供应商数量'] < min([result['最少供应商数量'] for result in mc_results.values()]):
        recommendations.append("推荐采用混合材料优化方案，可用更少供应商满足需求")
    
    if optimal_combo['成功率'] >= 0.95:
        recommendations.append("当前方案具有高置信度，风险可控")
    else:
        recommendations.append("建议适当增加供应商数量以提高供应安全性")
    
    if optimal_combo['A类供应商'] > optimal_combo['C类供应商']:
        recommendations.append("A类材料供应商较多，注意成本控制")
    
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
    
    return all_options

# 执行可视化和报告生成
print("\n正在生成可视化结果...")
visualize_simulation_results(mc_results, mixed_results)

print("\n正在生成分析报告...")
analysis_summary = generate_analysis_report(mc_results, mixed_results, valid_suppliers)

In [ ]:
# 保存分析结果
def save_simulation_results(mc_results, mixed_results, valid_suppliers):
    """保存蒙特卡洛模拟结果到Excel文件"""
    
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    
    with pd.ExcelWriter(f'DataFrames/蒙特卡洛模拟结果_{timestamp}.xlsx', engine='openpyxl') as writer:
        
        # 1. 总结页
        summary_data = []
        
        # 单一材料类型结果
        for material_type, result in mc_results.items():
            summary_data.append({
                '方案类型': f'{material_type}类单一供应',
                '最少供应商数量': result['最少供应商数量'],
                '成功率': result['最终成功率'],
                '总需求(立方米)': result['总需求'],
                '可选供应商数': result['可选供应商总数']
            })
        
        # 混合材料结果
        optimal_combo = mixed_results['最优组合']
        summary_data.append({
            '方案类型': '混合材料优化',
            '最少供应商数量': optimal_combo['供应商数量'],
            '成功率': optimal_combo['成功率'],
            '总需求(立方米)': '多类型混合',
            '可选供应商数': len(valid_suppliers)
        })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='结果总览', index=False)
        
        # 2. 混合方案详细信息
        mixed_detail = pd.DataFrame(mixed_results['所有组合'])
        mixed_detail.to_excel(writer, sheet_name='混合方案详情', index=False)
        
        # 3. 最优供应商列表
        optimal_suppliers = mixed_results['最优供应商列表'].copy()
        optimal_suppliers_output = optimal_suppliers[[
            '供应商名称', '材料类型', '平均供货量', '供货频率', 
            '供货稳定性', '风险调整因子', '综合评分'
        ]].round(4)
        optimal_suppliers_output.to_excel(writer, sheet_name='最优供应商列表', index=False)
        
        # 4. 各材料类型成功率序列
        max_len = max([len(result['成功率序列']) for result in mc_results.values()] + 
                     [len(mixed_results['成功率序列'])])
        
        success_rate_data = {}
        for material_type, result in mc_results.items():
            rates = result['成功率序列'] + [None] * (max_len - len(result['成功率序列']))
            success_rate_data[f'{material_type}类成功率'] = rates
        
        # 混合方案成功率
        mixed_rates = mixed_results['成功率序列'] + [None] * (max_len - len(mixed_results['成功率序列']))
        success_rate_data['混合方案成功率'] = mixed_rates
        success_rate_data['供应商数量'] = list(range(1, max_len + 1))
        
        success_rate_df = pd.DataFrame(success_rate_data)
        success_rate_df.to_excel(writer, sheet_name='成功率变化', index=False)
    
    print(f"✓ 结果已保存到: DataFrames/蒙特卡洛模拟结果_{timestamp}.xlsx")
    return f'DataFrames/蒙特卡洛模拟结果_{timestamp}.xlsx'

# 最终结论和建议
def final_conclusion(mc_results, mixed_results):
    """输出最终结论和建议"""
    
    print("\n" + "="*60)
    print("🎯 最终结论与建议")
    print("="*60)
    
    optimal_combo = mixed_results['最优组合']
    
    print(f"\n【推荐方案】混合材料优化方案")
    print(f"┌─ 最少供应商数量: {optimal_combo['供应商数量']}家")
    print(f"├─ 供应成功率: {optimal_combo['成功率']:.1%}")
    print(f"├─ 供应商构成:")
    print(f"│  ├─ A类材料: {optimal_combo['A类供应商']}家")
    print(f"│  ├─ B类材料: {optimal_combo['B类供应商']}家")
    print(f"│  └─ C类材料: {optimal_combo['C类供应商']}家")
    print(f"└─ 总供应能力: {optimal_combo['平均供应能力']:,.0f} 立方米")
    
    print(f"\n【方案优势】")
    advantages = []
    
    # 与单一材料方案比较
    single_material_min = min([result['最少供应商数量'] for result in mc_results.values()])
    if optimal_combo['供应商数量'] <= single_material_min:
        advantages.append(f"供应商数量最少，仅需{optimal_combo['供应商数量']}家")
    
    if optimal_combo['成功率'] >= 0.95:
        advantages.append("高置信度保障（≥95%）")
    
    if optimal_combo['A类供应商'] + optimal_combo['B类供应商'] > optimal_combo['C类供应商']:
        advantages.append("优先使用高效率材料，降低原材料消耗")
    
    advantages.append("分散供应风险，避免单一材料依赖")
    
    for i, advantage in enumerate(advantages, 1):
        print(f"  {i}. {advantage}")
    
    print(f"\n【实施建议】")
    suggestions = [
        f"立即与前{optimal_combo['供应商数量']}家综合评分最高的供应商建立合作关系",
        "建立供应商绩效监控体系，定期评估供货稳定性",
        "制定应急预案，准备2-3家备选供应商",
        "根据实际执行情况，每季度重新评估供应商组合",
        "优化库存管理策略，平衡各类材料的库存比例"
    ]
    
    for i, suggestion in enumerate(suggestions, 1):
        print(f"  {i}. {suggestion}")
    
    print(f"\n【风险提示】")
    risks = []
    
    if optimal_combo['成功率'] < 0.98:
        risks.append(f"当前成功率为{optimal_combo['成功率']:.1%}，建议考虑增加1-2家备选供应商")
    
    if optimal_combo['A类供应商'] > 5:
        risks.append("A类材料供应商较多，注意控制采购成本（比C类高20%）")
    
    if optimal_combo['C类供应商'] < 3:
        risks.append("C类材料供应商较少，可能存在供应集中风险")
    
    if not risks:
        risks.append("当前方案风险较低，建议按计划执行")
    
    for i, risk in enumerate(risks, 1):
        print(f"  {i}. {risk}")

# 执行结果保存和最终总结
print("\n正在保存分析结果...")
saved_file = save_simulation_results(mc_results, mixed_results, valid_suppliers)

print("\n正在生成最终结论...")
final_conclusion(mc_results, mixed_results)

print(f"\n🎉 蒙特卡洛模拟分析完成!")
print(f"📁 详细结果已保存到: {saved_file}")
print(f"📊 可视化图表已保存到: Pictures/monte_carlo_simulation_results.svg")